# Agenda, week 2


1. Recap and Q&A
2. dtypes in Pandas
     - What are they?
     - How do they work?
     - How do we change them?
     - Why do we care?
3. `NaN` -- "not a number"
    - What is it?
    - Why do we need it?
    - How do we work with it?
4. Data frames    
    - Creating data frames
    - Retrieving from them (rows vs. columns)
    - `.loc` and `.iloc`
5. Adding and removing data
    - Add rows
    - Add columns
    - Remove rows
    - Remove columns
6. Useful methods and attributes    
7. Using boolean ("mask") indexes to retrieve interesting data
    - Using `.loc` with a row 